In [1]:
import os
from pathlib import Path

from sklearn.metrics import accuracy_score, f1_score

from bella.parsers import semeval_14
from bella.helper import read_config
from bella.tokenisers import ark_twokenize

Using TensorFlow backend.


# How to use the pre-trained models

A lot of the models trained in the model zoo took a long time to train espically the Neural Network models which include LSTM, TDLSTM, and TCLSTM. In this notebook we are going to show you how to:
1. Load the models
2. Use the models to predict on some of the test data

First we need the path to the model zoo:

In [2]:
# Change this to the directory of your model zoo
model_zoo_dir = os.path.abspath('../model zoo')
os.listdir(model_zoo_dir)

['TCLSTM Election Twitter weights',
 'TDParse Mitchel',
 'LSTM Dong Twitter weights',
 'TDParse SemEval 14 Laptop',
 'TCLSTM SemEval 14 Restaurant weights',
 'TDLSTM SemEval 14 Restaurant weights',
 'TDLSTM SemEval 14 Restaurant architecture',
 'TDParse Plus YouTuBean',
 'LSTM SemEval 14 Restaurant weights',
 'Target Dependent Dong Twitter',
 'Target Dependent Plus Mitchel',
 'TDParse Election Twitter',
 'LSTM YouTuBean weights',
 'TDParse Plus Mitchel',
 'TDParse Plus SemEval 14 Laptop',
 'TDLSTM Mitchel weights',
 'TCLSTM YouTuBean architecture',
 'Target Dependent Mitchel',
 'Target Dependent Plus SemEval 14 Laptop',
 'TCLSTM YouTuBean weights',
 'LSTM Mitchel architecture',
 'TDLSTM YouTuBean weights',
 'TDLSTM Mitchel architecture',
 'TDParse Plus Election Twitter',
 'Target Dependent Plus SemEval 14 Restaurant',
 'Target Dependent SemEval 14 Laptop',
 'Target Dependent Plus YouTuBean',
 'LSTM YouTuBean architecture',
 'TDParse Plus Dong Twitter',
 'TCLSTM Dong Twitter architectur

In the model zoo you should be able to see the list of models as shown above. The models in there can be broken up into different methods which are the following: 1. Target Dependent, 2. Target Dependent Plus, 3. TDParse, 4. TDParse Plus, 5. LSTM, 6. TDLSTM, 7. TCLSTM. Each one of these methods was trained on the datasets which can be found in the [Dataset notebook](./datasets.ipynb). Therefore we have a model for each method on each dataset, if you would like to see how these models were trained look at the following Mass Evaluation notebooks: 1. [Target Dependent methods](./Mass Evaluation - Target Dependent.ipynb), 2. [TDParse methods](./Mass Evaluation - TDParse.ipynb), 3. [LSTM methods](./Mass Evaluation LSTM.ipynb)

We are going to show how to load a model from each of the methods on the [SemEval 2014 Resturant dataset](http://alt.qcri.org/semeval2014/task4/index.php?id=data-and-tools).

We are going to load one model at a time as each model can take:
1. A long time to load
2. Take up a lot of memory

For each model we will:
1. Load it
2. Predict the sentiment of the test data on the SemEval 2014 Resturant dataset
3. Evaluate the performance using Accuracy and Macro F1 score.

But before going through each model we need to load the test data:

In [3]:
CONFIG_FP = Path('..', 'config.yaml')

train_data = semeval_14(read_config('semeval_2014_rest_train', CONFIG_FP))
test_data = semeval_14(read_config('semeval_2014_rest_test', CONFIG_FP))
y_test = test_data.sentiment_data()
# Need to load the word vectors that were used to train the LSTM models 
# therefore we need to filter the word vectors down to only the train and test 
# dataset words as these were used during training the model because if you 
# load the word vectors whole it will use a lot of memory up to ~6GB
train_words = train_data.word_list(ark_twokenize)
test_words = test_data.word_list(ark_twokenize)
all_words = train_words + test_words

As the test data has been loaded all the cells grouped by models can be run independently.

## Target Dependent and Target Dependent Plus Models

These two methods come from [Vo and Zhang's paper](https://www.ijcai.org/Proceedings/15/Papers/194.pdf).

In [4]:
from bella.models.target import TargetDep

target_dep = TargetDep()
model_path = [model_zoo_dir, 
              'Target Dependent SemEval 14 Restaurant']
target_dep_model_path = os.path.join(*model_path)
# Loads the model
target_dep.load_model(target_dep_model_path, verbose=1)
# Predicts the class on the test data
target_dep_preds = target_dep.predict(test_data.data())
# Evaluates the results
target_dep_f1 = f1_score(y_test, target_dep_preds, average='macro')
target_dep_acc = accuracy_score(y_test, target_dep_preds)
print('Target Dependent scores:\nMacro F1 score: {}\n'
      'Accuracy: {}'
      .format(target_dep_f1 * 100, target_dep_acc * 100))

Loading model from /home/andrew/Desktop/Bella/model zoo/Target Dependent SemEval 14 Restaurant
Model successfully loaded. Load time 0.63
Target Dependent scores:
Macro F1 score: 56.15868320061148
Accuracy: 73.83928571428572


In [5]:
from bella.models.target import TargetDepSent

target_dep_plus = TargetDepSent()
model_path = [model_zoo_dir, 
              'Target Dependent Plus SemEval 14 Restaurant']
target_dep_plus_model_path = os.path.join(*model_path)
# Loads the model
target_dep_plus.load_model(target_dep_plus_model_path, verbose=1)
# Predicts the class on the test data
target_dep_plus_preds = target_dep_plus.predict(test_data.data())
# Evaluates the results
target_dep_plus_f1 = f1_score(y_test, target_dep_plus_preds, average='macro')
target_dep_plus_acc = accuracy_score(y_test, target_dep_plus_preds)
print('Target Dependent Plus scores:\nMacro F1 score: {}\n'
      'Accuracy: {}'
      .format(target_dep_plus_f1 * 100, target_dep_plus_acc * 100))


Loading model from /home/andrew/Desktop/Bella/model zoo/Target Dependent Plus SemEval 14 Restaurant
Model successfully loaded. Load time 0.69
Target Dependent Plus scores:
Macro F1 score: 57.71114226900386
Accuracy: 74.64285714285714


## TDParse and TDParse Plus Models

These two methods come from [Wang et al.'s paper](https://www.aclweb.org/anthology/E17-1046). Just a note that these models use the Tweebo dependency parser. If they are not working this could be due to Tweebo not being installed correctly.

In [6]:
from bella.models.tdparse import TDParse

tdparse = TDParse()
model_path = [model_zoo_dir, 
              'TDParse SemEval 14 Restaurant']
tdparse_model_path = os.path.join(*model_path)
# Loads the model
tdparse.load_model(tdparse_model_path, verbose=1)
# Predicts the class on the test data
tdparse_preds = tdparse.predict(test_data.data())
# Evaluates the results
tdparse_f1 = f1_score(y_test, tdparse_preds, average='macro')
tdparse_acc = accuracy_score(y_test, tdparse_preds)
print('TDParse scores:\nMacro F1 score: {}\n'
      'Accuracy: {}'
      .format(tdparse_f1 * 100, tdparse_acc * 100))

Loading model from /home/andrew/Desktop/Bella/model zoo/TDParse SemEval 14 Restaurant
Model successfully loaded. Load time 0.69
TDParse scores:
Macro F1 score: 59.49185457979117
Accuracy: 73.39285714285714


In [7]:
from bella.models.tdparse import TDParsePlus

tdparse_plus = TDParsePlus()
model_path = [model_zoo_dir, 
              'TDParse Plus SemEval 14 Restaurant']
tdparse_plus_model_path = os.path.join(*model_path)
# Loads the model
tdparse_plus.load_model(tdparse_plus_model_path, verbose=1)
# Predicts the class on the test data
tdparse_plus_preds = tdparse_plus.predict(test_data.data())
# Evaluates the results
tdparse_plus_f1 = f1_score(y_test, tdparse_plus_preds, average='macro')
tdparse_plus_acc = accuracy_score(y_test, tdparse_plus_preds)
print('TDParse Plus scores:\nMacro F1 score: {}\n'
      'Accuracy: {}'
      .format(tdparse_plus_f1 * 100, tdparse_plus_acc * 100))

Loading model from /home/andrew/Desktop/Bella/model zoo/TDParse Plus SemEval 14 Restaurant
Model successfully loaded. Load time 0.7
TDParse Plus scores:
Macro F1 score: 60.979583726818774
Accuracy: 76.16071428571428


## LSTM, TDLSTM and TCLSTM Models

These three methods come from [Tang et al.'s paper](https://aclweb.org/anthology/C16-1311). These models do require you knowing what embeddings they were trained on, this can only be done currently by looking at the relevent methods word vector results file e.g. for the LSTM method the word vector results file can be found [here](../results/TDLstm/lstm/word vector results.json). **NOTE** all the methods were trained using the Ark Tokeniser which is a Twitter specific tokeniser and all words lower cased

**NOTE** Padding value when loading the word vectors means that when converting words into word vectors if there is a word that is not in the word embedding it gets that embedding value of <unk> which in the case of SSWE has been learnt and the value of the padding vectors will be equal to the Padding value. If this is not set then the value of the unknown vectors will be used for both padding and unknown words. **We are not in this case going to use the padding value as we did not use it while training the networks**
    
All the neural network models require two file paths: 
1. To the network architecture
2. To the network's weights

In [8]:
from bella.models.tdlstm import LSTM
from bella.word_vectors import SSWE

# Load the SSWE word vector
sswe = SSWE(filter_words=all_words)

lstm = LSTM(tokeniser=ark_twokenize, embeddings=sswe, lower=True)
model_arch_path = [model_zoo_dir, 
                   'LSTM SemEval 14 Restaurant architecture']
model_weights_path = [model_zoo_dir, 
                      'LSTM SemEval 14 Restaurant weights']
# Loads the model
lstm.load_model(model_arch_fp=os.path.join(*model_arch_path), 
                model_weights_fp=os.path.join(*model_weights_path), 
                verbose=1)
lstm.test_pad_size = 78
# Predicts the class on the test data
sentiment_mapper = {0 : 0, 1 : 1, 2 : -1}

lstm_preds = lstm.predict(test_data.data_dict())
lstm_preds =  lstm.prediction_to_cats(y_test, lstm_preds, 
                                      mapper=sentiment_mapper)
# Evaluates the results
lstm_f1 = f1_score(y_test, lstm_preds, average='macro')
lstm_acc = accuracy_score(y_test, lstm_preds)
print('LSTM scores:\nMacro F1 score: {}\n'
      'Accuracy: {}'
      .format(lstm_f1 * 100, lstm_acc * 100))

Model architecture loaded /home/andrew/Desktop/Bella/model zoo/LSTM SemEval 14 Restaurant architecture
Model weights loaded /home/andrew/Desktop/Bella/model zoo/LSTM SemEval 14 Restaurant weights
Load time 0.29
LSTM scores:
Macro F1 score: 48.7152129456744
Accuracy: 73.30357142857142


/home/andrew/Envs/Bella/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [9]:
from bella.models.tdlstm import TDLSTM
from bella.word_vectors import GloveCommonCrawl

# Load the Glove word vector
glove_300 = GloveCommonCrawl(version=42, filter_words=all_words)

tdlstm = TDLSTM(tokeniser=ark_twokenize, embeddings=glove_300, lower=True)
model_arch_path = [model_zoo_dir, 
                   'TDLSTM SemEval 14 Restaurant architecture']
model_weights_path = [model_zoo_dir, 
                      'TDLSTM SemEval 14 Restaurant weights']
# Loads the model
tdlstm.load_model(model_arch_fp=os.path.join(*model_arch_path), 
                  model_weights_fp=os.path.join(*model_weights_path), 
                  verbose=1)
# Predicts the class on the test data
sentiment_mapper = {0 : 0, 1 : 1, 2 : -1}

tdlstm_preds = tdlstm.predict(test_data.data_dict())
tdlstm_preds =  tdlstm.prediction_to_cats(y_test, tdlstm_preds, 
                                          mapper=sentiment_mapper)
# Evaluates the results
tdlstm_f1 = f1_score(y_test, tdlstm_preds, average='macro')
tdlstm_acc = accuracy_score(y_test, tdlstm_preds)
print('TDLSTM scores:\nMacro F1 score: {}\n'
      'Accuracy: {}'
      .format(tdlstm_f1 * 100, tdlstm_acc * 100))

Loading glove 300d 42b common crawl from file
Model architecture loaded /home/andrew/Desktop/Bella/model zoo/TDLSTM SemEval 14 Restaurant architecture
Model weights loaded /home/andrew/Desktop/Bella/model zoo/TDLSTM SemEval 14 Restaurant weights
Load time 0.74
TDLSTM scores:
Macro F1 score: 59.833699184621736
Accuracy: 74.46428571428572


In [10]:
from bella.models.tdlstm import TCLSTM
from bella.word_vectors import GloveCommonCrawl

# Load the Glove word vector
glove_300 = GloveCommonCrawl(version=42, filter_words=all_words)

tclstm = TCLSTM(tokeniser=ark_twokenize, embeddings=glove_300, lower=True)
model_arch_path = [model_zoo_dir, 
                   'TCLSTM SemEval 14 Restaurant architecture']
model_weights_path = [model_zoo_dir, 
                      'TCLSTM SemEval 14 Restaurant weights']
# Loads the model
tclstm.load_model(model_arch_fp=os.path.join(*model_arch_path), 
                        model_weights_fp=os.path.join(*model_weights_path), 
                        verbose=1)
# Predicts the class on the test data
sentiment_mapper = {0 : 0, 1 : 1, 2 : -1}

tclstm_preds = tclstm.predict(test_data.data_dict())
tclstm_preds =  tclstm.prediction_to_cats(y_test, tclstm_preds, 
                                                mapper=sentiment_mapper)
# Evaluates the results
tclstm_f1 = f1_score(y_test, tclstm_preds, average='macro')
tclstm_acc = accuracy_score(y_test, tclstm_preds)
print('TCLSTM scores:\nMacro F1 score: {}\n'
      'Accuracy: {}'
      .format(tclstm_f1 * 100, tclstm_acc * 100))

Loading glove 300d 42b common crawl from file
Model architecture loaded /home/andrew/Desktop/Bella/model zoo/TCLSTM SemEval 14 Restaurant architecture
Model weights loaded /home/andrew/Desktop/Bella/model zoo/TCLSTM SemEval 14 Restaurant weights
Load time 2.04
TCLSTM scores:
Macro F1 score: 49.58447514820157
Accuracy: 68.39285714285714
